In [2]:
from src.data.dataset_functions import get_dataframei_onlymods

mod_dataframe = get_dataframei_onlymods(1) #Der Parameter steht für den Datensatz, der verwendet werden soll

providing ../data/raw/2022_place_canvas_history-000000000001.csv ...
../data/raw/2022_place_canvas_history-000000000001.csv is already in data/raw
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-add7f80d-d57a-4974-af8a-753c7488e812;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 78ms :: artifacts dl 15ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------

<div style="color:#246fd6">
Im folgenden sollen die Statistiken für Mods bestimmt werden:


1) Wie oft mussten Mods zensieren im jeweiligen Datensatz?

2) Wie viele Pixel wurden insgesamt von einem Mod im jeweiligen Datensatz zensiert?

3) Werden Regionen deutlich, in denen Mods häufiger eingreifen mussten?

    3.1) Wenn ja, welche Regionen und welchen Artworks ist das zugeordnet?

    3.2) Zu bestimmen, welche Pixel mehr als einmal zensiert wurden, also min. 2-mal.

4) Evtl.: Wurden von Bots gesetzte Pixel zensiert?

</div>

<div style="color:#246fd6">
Zur Beantwortung der ersten Frage:

Mods sind im "StandardDataFrame" nicht enthalen, nur noch im spezifischen ModsDataFrame!

Um die Frage zu beantworten muss gezählt werden, wie viele Einträge für Mods im jeweiligen Datensatz exitieren.
D.h. auch Duplikate von User_IDs sind relevant, sodass einfach nur gezählt werden muss, wie viele Einträge insgesamt im ModsDataFrame existieren.

Das wäre dann auch schon die Antwort darauf, wie oft Mods im jeweiligen Datensatz zensieren mussten.
</div>

In [3]:
amount_of_censorship = mod_dataframe.count()
print(amount_of_censorship)

1


<div style="color:#246fd6">
Beantwortung der 2. Frage:
Betrachtung der zensierten Pixel pro Mod.

D.h. hier muss jeder Mod einzeln betrachtet werden und dabei noch berechnet werden, wie viele Pixel dabei das Quadrat aufgespannt haben.

Berechnung der Pixel: 
x1,y1 => obere linke Ecke des Quadrates 
x2,y2 => untere rechte Ecke des Quadrates

Damit ergibt sich:
```
 x1,y1----------------------x2,y1
      |                    |
      |                    |
      |                    |
    a |                    |
      |                    |
      |                    |
      |                    |
 x1,y2----------------------x2,y2
                b
```
Dann ergibt a * b die Anzahl an Pixel, die dabei berührt wurden und somit auch die Anzahl der Pixel, die dabei zensiert wurden.

a = y2 - y1

b = x2 - x1

Anzahl der zensierten Pixel pro Eintrag = (y2 - y1) * (x2 - x1)
</div>

In [9]:
pixel_amount = mod_dataframe.withColumn("amount",(mod_dataframe.y2 - mod_dataframe.y1) * (mod_dataframe.x2 - mod_dataframe.x1))
pixel_amount.printSchema()
pixel_amount.show()

root
 |-- user_id: string (nullable = true)
 |-- x1: integer (nullable = true)
 |-- y1: integer (nullable = true)
 |-- x2: integer (nullable = true)
 |-- y2: integer (nullable = true)
 |-- t: long (nullable = true)
 |-- pixel_color: string (nullable = true)
 |-- amount: integer (nullable = true)

+--------------------+----+----+----+----+----------+-----------+------+
|             user_id|  x1|  y1|  x2|  y2|         t|pixel_color|amount|
+--------------------+----+----+----+----+----------+-----------+------+
|q+XjkQ6WRx0aBLtb2...|1349|1718|1424|1752|1649035370|    #000000|  2550|
+--------------------+----+----+----+----+----------+-----------+------+



<div style="color:#246fd6">
Nochmal über alle Mods summiert:
</div>

In [10]:
amounts = pixel_amount.agg({'amount': 'sum'}).show()

+-----------+
|sum(amount)|
+-----------+
|       2550|
+-----------+



<div style="color:#246fd6">
Beantwortung der 3.Frage:
Regionen bestimmen, in denen häufiger eingegriffen wurde.

Dafür Überschneidungen zwischen den Pixeln der Einträge bestimmen.

Zu beachten dabei: Es müssen alle von der Zensur berührten Pixel betrachtet werden bei der Überschneidung.
Es müssen verschiedene Fälle betrachtet werden:

1) Eins der beiden Quadrate liegt genau im anderen.

    Dann ist x1b < x1s & x2s < x2b & y1b < y1s & y2b > y2s

    s => Quadrat s das vollständig im Quadrat b liegt

2) Teilweise Überschneidungen.

    Hier wirds schwierig, denn hier dürfen nur die in der Überschneidung liegenden Gebiete als "Mehrfach zensiert" gelabelt werden.
</div>